In [3]:
# Basic imports
import scipy, pandas as pd, numpy as np
from matplotlib import pyplot as plt

In [124]:
# Zone to FIPS crosswalk
ztofips =pd.read_csv("ZonetoFIPS.txt", names=["STATE", "ZONE", "CWA", "NAME", "STATE_ZONE", "COUNTY", "FIPS", "TIME_ZONE", "FE_AREA", "LAT", "LON"], sep='|')

In [6]:
# state abbreviations and fips crosswalk
fipstoab = pd.read_csv("us-state-ansi-fips.csv")
fipstoab.rename(columns={' st': 'STATE_FIPS'}, inplace=True)

In [5]:
# testing of some missing NWS to FIPs crosswalks
mask = ztofips['STATE_ZONE'] == 'NY077'
ztofips[mask]

,STATE,ZONE,CWA,NAME,STATE_ZONE,COUNTY,FIPS,TIME_ZONE,FE_AREA,LAT,LON


In [125]:
fipstozips = pd.read_csv("fipstozips.csv")

In [121]:
def proc(noloc):
    # convert the datetime file back to datetime object from text
    noloc['beg_datetime'] = pd.to_datetime(noloc['beg_datetime'], format='%m/%d/%Y %H:%M:%S')

    zones = noloc[noloc['CZ_TYPE'] == ' Z']

    zones_ab = pd.merge(zones, fipstoab, on='STATE_FIPS', how='left')

    zones_ab.rename(columns={' stusps': 'stusps'}, inplace=True)
    zones_ab['stusps'] = zones_ab['stusps'].str.strip()
    
    notna_mask = zones_ab['stusps'].notna()
    zones_ab = zones_ab[notna_mask]

    # this stuff gives the setting with copy warning
    zones_ab['zone_padded'] = zones_ab['CZ_FIPS'].astype(str).str.zfill(3)
    zones_ab['STATE_ZONE'] = zones_ab['stusps'] + zones_ab['zone_padded']
    
    mask = zones_ab['STATE_ZONE'].notna()
    
    zones_fips = pd.merge(zones_ab, ztofips, on='STATE_ZONE', how='left')

    # There needs to be some sort of crosswalking between old and new NWS zones 
    # in order for the crosswalk file given to work. Right now we're missing some.

    # temp solution ignoring the NA values in order to write rest of the code
    notnamask = zones_fips['FIPS'].notna()
    zones_fips= zones_fips[notnamask]
    zones_fips['FIPS'] = zones_fips['FIPS'].astype(int)

    # for non zone entries with code C
    others = noloc[noloc['CZ_TYPE'] == ' C'].copy()
    others.loc[:, 'FIPS'] = others['STATE_FIPS'].astype(str) + others['CZ_FIPS'].astype(str).str.zfill(3)
    others['FIPS'] = others['FIPS'].astype(int)

    # combine
    merged = pd.concat([zones_fips, others])
    # crosswalk to zip
    merged = pd.merge(merged, fipstozips, on="FIPS")

    # cut columns
    columns = ["GEOID20", "F_", "FIPS", "EVENT_TYPE", "beg_datetime", "days"]
    filtered = merged.loc[:, columns]

    # index + sort
    rindex = filtered.set_index("GEOID20")
    sorted = rindex.sort_index(ascending=True)

    return sorted

In [127]:
years = range(2006, 2023)
columnNames = ["GEOID20", "EVENT_ID", "FIPS", "EVENT_TYPE", "beg_datetime", "days"]

for year in years:
    noloc = pd.read_csv("storm_noloc/storm" + str(year) + "_noloc.csv")
    result = proc(noloc)

    np.savetxt("storm" + str(year) + "_noloc_zips" + ".csv", result, fmt='%s', delimiter=', ', header=', '.join(columnNames))